<h1 align="center">
    YOLO v8-segmentation training 
</h1>

> ### This notebook contains the required code to train the YOLO v8 segmentation model and run its inference on a set of files
___

#### Imports used packages

In [1]:
# All imports needed
from ultralytics import YOLO
import os
import cv2 as cv
import numpy as np
from PIL import Image

#### Loads pre-trained YOLO v8-segmentation model

In [2]:
# Load a model 
# Checkpoint = v3 = "D:\\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\ResistorWizard\\Backend\\YOLO_Segmenter\\runs\\segment\\train\\weights\\best.pt"
# Checkpoint = v3 + v6 = "D:\\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\ResistorWizard\\Backend\\YOLO_Segmenter\\runs\\segment\\train2\\weights\\best.pt"
model = YOLO("D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\resistor-wizard\\Backend\\YOLO_Segmenter\\latest\\best.pt")

#### Trains the loaded model on the dataset

In [3]:
# Train model with custom dataset
# results = model.train(data="D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\ResistorWizard\\Backend\\YOLO_Segmenter\\Metal Film Leaded Resistor Color Bands v6\\data.yaml", epochs=100)

#### Runs inference on a set of files

In [4]:
# Returns a list of all files in a given directory
def get_files(dir):
    tests = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            tests.append(os.path.join(dir,name))
    return tests

files = get_files("")

In [122]:
path_test = "D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\resistor-wizard\\Backend\\Tests\\res2.png"
pred = model(path_test, save=True, conf=0.6, show_labels=False, show_conf=False, save_conf=True, iou=0.3)


image 1/1 D:\Joule\Documents\Jhony\Universidade\UTFPR\2023.2\Oficinas\resistor-wizard\Backend\Tests\res2.png: 640x640 4 Resistor-Color-Bandss, 1 Resistor-Color-Bands-Start, 130.0ms
Speed: 9.6ms preprocess, 130.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict7


In [123]:
boxes = pred[0].boxes.cpu().data
vertex = [v[:4] for v in boxes]
print(vertex[0])

tensor([458.2142, 282.1049, 528.2455, 450.5578])


In [124]:
print(pred[0].boxes.conf)
classes = pred[0].names
results_with_probs = [(result, classes[result.boxes.cpu().cls.numpy()[0]]) for result in pred[0]]

tensor([0.8828, 0.8774, 0.8707, 0.8647, 0.8473], device='cuda:0')


In [125]:
# Checks if two colors are equal
def eqq(a,b):
    if len(a) != len(b):
        return False
    for i in range(len(a)):
        if a[i] != b[i]:
                return False
    return True


class Mask:
    def __init__(self, img, bbox=[], contour=[]):
        self.image = img
        self.orig_shape = img.shape
        self.mask = np.zeros_like(img, dtype=np.uint8)  # creates mask of same size as original image
        self.size = -1  # Number of non-zero pixels
        self.avgColor = (None, None, None)
        self.bbox = bbox
        self.contour = contour
        self.index = -1  # This color's position relative to others
        self.color = ""  # A string identifier of the color the mask represents
    #
    def maskImage(self):
        pixels = np.zeros_like(self.image, dtype=np.uint8)
        for x in range(len(self.image)):
            for y in range(len(self.image[0])):
                if eqq(self.mask[x][y], [255,255,255]):
                    pixels[x][y] = self.image[x][y]
        return pixels
    #
    def sample_avg_color(self):
        pixels = self.maskImage()
        self.avgColor = np.sum(pixels.reshape((-1,1,3)), axis=0) / self.size
        
    # Checks if a point is inside the mask's bounding box
    def contains(self, point):
        if list(point) < self.bbox[0:2].tolist() or list(point) > self.bbox[2:4].tolist():
            return False
        else:
            return True

In [126]:
# Calculates the centroid of a rectangle
def get_centroid(vertex):
    cX = (vertex[2] + vertex[0])/2
    cY = (vertex[3] + vertex[1])/2
    return (cX, cY)

# Calculates the linear regression of the points in a 2D dataset: Y in function of X (Y= a*X + b)
def lin_reg(data):
    X = np.array([i[0] for i in data])
    Y = np.array([i[1] for i in data])
    a = ((len(data) * np.sum(X*Y)) - (np.sum(X) * np.sum(Y)))/(len(data) * np.sum(X*X) - (np.sum(X))**2)
    b = ((np.sum(Y) * np.sum(X*X)) - (np.sum(X) * np.sum(X*Y)))/(len(data) * np.sum(X*X) - (np.sum(X))**2)
    return a,b

# Applies a linear equation on a point (Y = a*X + b)
def f(x, params):
    return params[0]*x + params[1]

# Retrieves the pixels contained in the contours given by the model prediction
def get_segmentation_masks(img, inference, data=255):
    image = cv.imread(img)
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    masks = [Mask(image) for i in range(len(inference[0].masks.xy))]
    #
    for i in range(len(inference[0].masks.xy)):
        polygon = inference[0].masks.xy[i].astype(int)
        masks[i].bbox = inference[0].boxes.cpu().data[i]
        masks[i].contour = polygon.reshape((-1, 1, 2))  # Reshape to match the format expected by cv.drawContours
        masks[i].mask = cv.fillPoly(masks[i].mask, [masks[i].contour], (data, data, data)) # converts polygon points to pixels inside it
        masks[i].size = 0
        for j in masks[i].mask.reshape((-1,1,3)):
            if not eqq(j[0], [0,0,0]):
                masks[i].size += 1
        masks[i].sample_avg_color()
    return masks

# Finds the mask that contains a given point – considers a point can belong to a single mask
def retrieve_point_in_mask(masks, point):
    for mask in masks:
        if mask.contains(point):
            return mask

# Calculates the possible orders of the color bands – assumes the bands are approximately
#     linear, which is reasonable given that resistors follow this standard
def order_masks(masks, inference):
    boxes = inference[0].boxes.cpu().data
    vertex = [v[:4] for v in boxes]
    centroids = [get_centroid(v) for v in vertex]
    #
    # Calculates the line that approximately crosses the centroids – it's used to find the order in which the bands are disposed
    parameters = lin_reg(centroids)
    lv = np.array([1,parameters[0]]) # Line Vector
    lv_norm = np.sqrt(sum(lv**2))
    #
    vects = {}
    for center in centroids:
        u = np.array(center) - np.array([0,parameters[1]])  # Calculates the vector from the origin of the line to the centroid
        proj = lv * np.dot(u,lv) / (lv_norm**2)  # Projects the centroid onto the line
        vects[retrieve_point_in_mask(masks, center)] = np.sqrt(np.sum(proj**2))  # Calculates each centroid's distance to origin and associates it to its respective mask
    # Sorts masks by distance
    sorted_masks = dict(sorted(vects.items(), key=lambda item: item[1]))
    return list(sorted_masks)

In [127]:
# Retrieves the color name from masks average color
# Numbers on the side represent the number of samples used for each color's average
#  Files used to sample: res2.png, res.png, crop.jpg
COLOR_RANGES = {  # ~ UPDATE this with better values after installing camera ~
      "BLACK": (108, 115, 89),  # 2
      "BROWN": (103, 95, 104),   # 4
        "RED": (170, 173, 92),   # 2
     "ORANGE": (16, 218, 164),   # 2
     "YELLOW": (),
      "GREEN": (57, 95, 169),    # 1
       "BLUE": (),
     "VIOLET": (116, 138, 172),  # 1
       "GREY": (),
      "WHITE": (),
       "GOLD": (23, 72, 132),    # 2
     "SILVER": (),
}
def sample_color_names(mask):
    pass

In [128]:
masks = get_segmentation_masks(path_test, pred)

In [129]:
masks

In [130]:
ordered = order_masks(masks, pred)
print(len(ordered))

5


In [132]:
for i in range(len(ordered)):
    print(f"BBOX: {ordered[i].bbox[:4]} \t\t HSV: {ordered[i].avgColor}")

BBOX: tensor([163.2252, 347.4673, 237.1933, 539.1973]) 		 HSV: [[     28.068      83.247      172.79]]
BBOX: tensor([270.7560, 331.5136, 342.7377, 500.5215]) 		 HSV: [[     17.103      106.41      103.37]]
BBOX: tensor([351.3311, 351.6198, 427.5934, 478.7453]) 		 HSV: [[     109.12      86.364      61.703]]
BBOX: tensor([458.2142, 282.1049, 528.2455, 450.5578]) 		 HSV: [[     14.965      214.58      173.12]]
BBOX: tensor([537.4949, 244.9370, 608.6976, 440.3320]) 		 HSV: [[     17.431      220.44       155.4]]


In [15]:
image = cv.imread(path_test)
image_with_contours = cv.addWeighted(image, 1, m[0], 0.5, 0)  # Adjust opacity as needed
Image.fromarray(m[0]).save("D:\\Joule\\Documents\\Jhony\\Universidade\\UTFPR\\2023.2\\Oficinas\\resistor-wizard\\Backend\\Tests\\res_contour.png")

NameError: name 'm' is not defined

### Uses the trained model and a Convex Hull algorithm to auto-label new images for the dataset
> #### Reduces workload and speeds up labelling